<a href="https://colab.research.google.com/github/agarr3/vajra/blob/master/XlNetCapsule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=d4449039140ba390fb85725d82d6675f9ee4921153cfba906f826b833f942584
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 26.3 GB  |     Proc size: 118.4 MB
GPU RAM Free: 16280MB | Used: 0MB | Util   0% | Total     16280MB


In [2]:
!pip install transformers
!pip install gensim
#!pip install unidecode
!pip install sentencepiece

     |████████████████████████████████| 2.3MB 14.3MB/s 
     |████████████████████████████████| 3.3MB 51.2MB/s 
     |████████████████████████████████| 901kB 60.6MB/s 
     |████████████████████████████████| 1.2MB 14.3MB/s 


In [3]:
import numpy as np
import torch
import os
import shutil


class EarlyStoppingAndCheckPointer:
    """Early stops the training if validation loss doesn't improve after a given patience."""

    def __init__(self, patience=7, verbose=False, delta=0, basedir='.', trace_func=print, epoch_level_save=False):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.basedir = basedir
        self.trace_func = trace_func
        self.modelCheckPointer = ModelCheckPointer()
        self.epoch_level_save = epoch_level_save

    def __call__(self, val_loss, model, optimizer, epoch, scheduler):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.modelCheckPointer.save_checkpoint({'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optim_dict' : optimizer.state_dict(),
                                'sched_dict' : scheduler.state_dict()},
                               is_best=True,
                               checkpoint=self.basedir, save_epoch = self.epoch_level_save, epoch=epoch)
            self.trace_func(
                f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
            self.val_loss_min = val_loss

        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            self.modelCheckPointer.save_checkpoint({'epoch': epoch + 1,
                                                    'state_dict': model.state_dict(),
                                                    'optim_dict': optimizer.state_dict(),
                                                    'sched_dict' : scheduler.state_dict()},
                                                   is_best=False,
                                                   checkpoint=self.basedir, save_epoch = self.epoch_level_save, epoch=epoch)
            if self.counter >= self.patience:
                self.early_stop = True
            self.trace_func(
                f'Validation loss did not decrease. The patience counter is ({self.counter} ).  Saving model as a resume checkpoint ...')
        else:
            self.best_score = score
            self.modelCheckPointer.save_checkpoint({'epoch': epoch + 1,
                                                    'state_dict': model.state_dict(),
                                                    'optim_dict': optimizer.state_dict(),
                                                    'sched_dict' : scheduler.state_dict()},
                                                   is_best=True,
                                                   checkpoint=self.basedir, save_epoch = self.epoch_level_save, epoch=epoch)
            self.counter = 0
            self.trace_func(
                f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
            self.val_loss_min = val_loss

    # def save_checkpoint(self, val_loss, model):
    #     '''Saves model when validation loss decrease.'''
    #     if self.verbose:
    #         self.trace_func(
    #             f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
    #     torch.save(model.state_dict(), self.basedir)
    #     self.val_loss_min = val_loss


class ModelCheckPointer:
    def __init__(self):
        pass

    def save_checkpoint(self, state, is_best, checkpoint,  save_epoch = False, epoch=0):
        """Saves model and training parameters at checkpoint + 'last.pth.tar'. If is_best==True, also saves
        checkpoint + 'best.pth.tar'
        Args:
            state: (dict) contains model's state_dict, may contain other keys such as epoch, optimizer state_dict
            is_best: (bool) True if it is the best model seen till now
            checkpoint: (string) folder where parameters are to be saved
        """
        filepath = os.path.join(checkpoint, 'last.pth.tar')
        if not os.path.exists(checkpoint):
            print("Checkpoint Directory does not exist! Making directory {}".format(checkpoint))
            os.mkdirs(checkpoint)
        else:
            print("Checkpoint Directory exists! ")
        torch.save(state, filepath)
        if is_best:
            shutil.copyfile(filepath, os.path.join(checkpoint, 'best.pth.tar'))
        if save_epoch:
            shutil.copyfile(filepath, os.path.join(checkpoint, 'epoch-{}.pth.tar'.format(epoch)))


    def load_checkpoint(self, checkpoint, model, device, optimizer=None, scheduler=None):
        """Loads model parameters (state_dict) from file_path. If optimizer is provided, loads state_dict of
        optimizer assuming it is present in checkpoint.
        Args:
            checkpoint: (string) filename which needs to be loaded
            model: (torch.nn.Module) model for which the parameters are loaded
            optimizer: (torch.optim) optional: resume optimizer from checkpoint
        """
        filepath = os.path.join(checkpoint, 'last.pth.tar')
        if not os.path.exists(filepath):
            raise ("File doesn't exist {}".format(filepath))
        checkpoint = torch.load(checkpoint)
        model.load_state_dict(checkpoint['state_dict'], map_location=device)

        if optimizer:
            optimizer.load_state_dict(checkpoint['optim_dict'])
        if scheduler:
            scheduler.load_state_dict(checkpoint['sched_dict'])

        return checkpoint['epoch']

    def loadBestModel(self,checkpoint, model, device):
        filepath = os.path.join(checkpoint, 'best.pth.tar')
        if not os.path.exists(filepath):
            raise ("File doesn't exist {}".format(filepath))
        checkpoint = torch.load(filepath, map_location=device)
        model.load_state_dict(checkpoint['state_dict'])

class CustomLossUtils:
    def __init__(self):
        pass

    def getL1RegularizationLossTerm(self, model, params=None):
        regularization_loss = 0
        if params is None:
            for param in model.parameters():
                regularization_loss += torch.sum(torch.abs(param))
        else:
            for param in params:
                regularization_loss += torch.sum(torch.abs(param))

import torch
import torch.nn as nn
import torch.nn.functional as F

ALPHA = 0.8
GAMMA = 2


class FocalLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(FocalLoss, self).__init__()

    def forward(self, inputs, targets, alpha=ALPHA, gamma=GAMMA, smooth=1):
        # comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)

        # flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        # first compute binary cross-entropy
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        BCE_EXP = torch.exp(-BCE)
        focal_loss = alpha * (1 - BCE_EXP) ** gamma * BCE

        return focal_loss

In [14]:

from gensim.parsing.preprocessing import preprocess_documents, strip_tags, \
    strip_punctuation, strip_numeric, remove_stopwords, strip_short, stem_text

from gensim.parsing.preprocessing import strip_multiple_whitespaces
import unicodedata
import re

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, ' ', text)
    return text

def cleanSingleSentenceWithoutRemovingMeaning(x, minsize=3):
    processedx = strip_tags(x)
    #processedx = unidecode.unidecode(processedx)
    processedx = remove_accented_chars(processedx)
    processedx = remove_special_characters(processedx)
    processedx = strip_multiple_whitespaces(processedx)
    processedx = strip_numeric(processedx)
    processedx = strip_short(processedx, minsize=minsize)
    return processedx


def transformSingle(x):
    processedx = strip_tags(x)
    processedx = strip_punctuation(processedx)
    processedx = strip_multiple_whitespaces(processedx)
    processedx = strip_numeric(processedx)
    processedx = remove_stopwords(processedx)
    processedx = strip_short(processedx)
    #processedx = lemmatize_sentence(processedx)
    processedx = remove_accented_chars(processedx)
    processedx = remove_special_characters(processedx)
    return processedx.lower()

In [5]:
import math
import numpy as np

import torch


def squash(inputs, axis=-1):
    """
    The non-linear activation used in Capsule. It drives the length of a large vector to near 1 and small vector to 0
    :param inputs: vectors to be squashed
    :param axis: the axis to squash
    :return: a Tensor with same size as inputs
    """
    norm = torch.norm(inputs, p=2, dim=axis, keepdim=True)
    scale = norm**2 / (1 + norm**2) / (norm + 1e-8)
    return scale * inputs


import torch
from torch.autograd import Variable
import torch.nn.functional as F


class DenseCapsule(torch.nn.Module):
    """
    The dense capsule layer. It is similar to Dense (FC) layer. Dense layer has `in_num` inputs, each is a scalar, the
    output of the neuron from the former layer, and it has `out_num` output neurons. DenseCapsule just expands the
    output of the neuron from scalar to vector. So its input size = [None, in_num_caps, in_dim_caps] and output size = \
    [None, out_num_caps, out_dim_caps]. For Dense Layer, in_dim_caps = out_dim_caps = 1.

    :param in_num_caps: number of cpasules inputted to this layer
    :param in_dim_caps: dimension of input capsules
    :param out_num_caps: number of capsules outputted from this layer
    :param out_dim_caps: dimension of output capsules
    :param routings: number of iterations for the routing algorithm
    """
    def __init__(self, in_num_caps, in_dim_caps, out_num_caps, out_dim_caps, routings=3):
        super(DenseCapsule, self).__init__()
        self.USE_CUDA = torch.cuda.is_available()
        self.in_num_caps = in_num_caps
        self.in_dim_caps = in_dim_caps
        self.out_num_caps = out_num_caps
        self.out_dim_caps = out_dim_caps
        self.routings = routings
        self.weight = torch.nn.Parameter(0.01 * torch.randn(out_num_caps, in_num_caps, out_dim_caps, in_dim_caps))

    def forward(self, x):
        # x.size=[batch, in_num_caps, in_dim_caps]
        # expanded to    [batch, 1,            in_num_caps, in_dim_caps,  1]
        # weight.size   =[       out_num_caps, in_num_caps, out_dim_caps, in_dim_caps]
        # torch.matmul: [out_dim_caps, in_dim_caps] x [in_dim_caps, 1] -> [out_dim_caps, 1]
        # => x_hat.size =[batch, out_num_caps, in_num_caps, out_dim_caps]
        x_hat = torch.squeeze(torch.matmul(self.weight, x[:, None, :, :, None]), dim=-1)

        # In forward pass, `x_hat_detached` = `x_hat`;
        # In backward, no gradient can flow from `x_hat_detached` back to `x_hat`.
        x_hat_detached = x_hat.detach()

        # The prior for coupling coefficient, initialized as zeros.
        # b.size = [batch, out_num_caps, in_num_caps]
        if self.USE_CUDA:
            b = Variable(torch.zeros(x.size(0), self.out_num_caps, self.in_num_caps)).cuda()
        else:
            b = Variable(torch.zeros(x.size(0), self.out_num_caps, self.in_num_caps))

        assert self.routings > 0, 'The \'routings\' should be > 0.'
        for i in range(self.routings):
            # c.size = [batch, out_num_caps, in_num_caps]
            c = F.softmax(b, dim=1)

            # At last iteration, use `x_hat` to compute `outputs` in order to backpropagate gradient
            if i == self.routings - 1:
                # c.size expanded to [batch, out_num_caps, in_num_caps, 1           ]
                # x_hat.size     =   [batch, out_num_caps, in_num_caps, out_dim_caps]
                # => outputs.size=   [batch, out_num_caps, 1,           out_dim_caps]
                outputs = squash(torch.sum(c[:, :, :, None] * x_hat, dim=-2, keepdim=True))
                # outputs = squash(torch.matmul(c[:, :, None, :], x_hat))  # alternative way
            else:  # Otherwise, use `x_hat_detached` to update `b`. No gradients flow on this path.
                outputs = squash(torch.sum(c[:, :, :, None] * x_hat_detached, dim=-2, keepdim=True))
                # outputs = squash(torch.matmul(c[:, :, None, :], x_hat_detached))  # alternative way

                # outputs.size       =[batch, out_num_caps, 1,           out_dim_caps]
                # x_hat_detached.size=[batch, out_num_caps, in_num_caps, out_dim_caps]
                # => b.size          =[batch, out_num_caps, in_num_caps]
                b = b + torch.sum(outputs * x_hat_detached, dim=-1)

        return torch.squeeze(outputs, dim=-2)




In [6]:
import gc
import math
import os
import random

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.preprocessing import LabelBinarizer
from transformers import LongformerTokenizer, LongformerConfig, get_linear_schedule_with_warmup, BertConfig, \
    BertTokenizer, BertModel, XLNetConfig, XLNetTokenizer, XLNetModel

from transformers import BertForMaskedLM

import joblib
import torch
from torch import cuda


import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.sampler import SequentialSampler
from torch.hub import tqdm

#from pytorchtools import EarlyStoppingAndCheckPointer, ModelCheckPointer
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers.modeling_utils import SequenceSummary

In [7]:
class BertEnsembleModelConfig:
    defaultConfig = XLNetConfig()
    model_name = 'xlnet-base-cased'
    #model_name = "xlnet-large-cased"
    labelEncoderFileName = 'labelEncoder_xlnet_mood.sav'
    savedModelFileName = 'Bert_Ensemble_Model_v1.pt'
    tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
    if model_name == 'xlnet-base-cased':
        MAX_LEN = 1024
        TRAIN_BATCH_SIZE = [2, 2, 2, 2, 1, 1, 1, 1, 1]
        LERANING_RATE_DECAY_MANUAL = [1, 0.9, 0.9*0.9*0.9, 0.9*0.9*0.9*0.9, 0.9*0.9*0.9*0.9*0.9, 0.9*0.9*0.9*0.9*0.9*0.9, 0.9*0.9*0.9*0.9*0.9*0.9*0.9, 0.9*0.9*0.9*0.9*0.9*0.9*0.9, 0.9*0.9*0.9*0.9*0.9*0.9*0.9*0.9*0.9]
        ACCUMULATION_STEPS = 1
        PATIENCE = 9
        EPOCHS = 9
    elif model_name == "xlnet-large-cased":
        MAX_LEN = 1024
        TRAIN_BATCH_SIZE = [1, 1, 1, 1, 1]
        LERANING_RATE_DECAY_MANUAL = [1, 0.9, 0.9*0.9*0.9, 0.9*0.9*0.9*0.9, 0.9*0.9*0.9*0.9*0.9]
        ACCUMULATION_STEPS = 1
        PATIENCE = 3
        EPOCHS = 5
    MAX_LEN_FILENAME = 20
    VALID_BATCH_SIZE = 1
    

    LEARNING_RATE = 1e-05
    
    LEARNING_RATE_AUTO_DECAY_FLAG = False
    LR_DECAY_MODE = "EPOCH"

    WEIGHT_DECAY = 0.0
    
    WARM_UP_RATIO = 0.06
    WARM_UP_STEPS = 0
    max_grad_norm = None
    #LOSS_FN = "BCEWithLogitsLoss"
    #LOSS_FN = "CrossEntropyLoss"
    LOSS_FN = "MarginLoss"
    #BERT_MODEL_OP = "last_hidden"
    BERT_MODEL_OP = "CLS"

    VALID_FNAME_LEN_TH = 18
    HELD_OUT_VALIDATION = True
    CONDITIONAL_TRAINING = False

    DETERMINISTIC = True
    CAPS_PURE = True
    CLASS_LOSS_FN = "BCEWithLogitsLoss"
    #CLASS_LOSS_FN = "FocalLoss"
    if CAPS_PURE:
        LOSS_FN = "MarginLoss"
    else:
        LOSS_FN = CLASS_LOSS_FN

    USE_SEQUENCE_SUMMARY = True
    SQUASH_AFTER_TRANSFORMER = True
    SOFTMAX_DENSE_LAYER = False
    
    
  

In [10]:
class BertEnsembletModel(torch.nn.Module):
    '''
    classdocs
    '''

    def __init__(self, num_classes, configuration=None):
        super(BertEnsembletModel, self).__init__()
        if configuration:
            self.configuration = configuration
        else:
            self.configuration = BertEnsembleModelConfig()
        self.bert_model_content = XLNetModel.from_pretrained(self.configuration.model_name, mem_len=1024)

        xlNetDefConfig = XLNetConfig.from_pretrained(self.configuration.model_name)
        self.sequence_summary = SequenceSummary(xlNetDefConfig)

        if self.configuration.USE_SEQUENCE_SUMMARY:
            in_num_caps = self.configuration.MAX_LEN + 1
        else:
            in_num_caps = self.configuration.MAX_LEN

        if(self.configuration.model_name == "xlnet-large-cased"):
            self.digitcaps = DenseCapsule(in_num_caps=in_num_caps,
                                          in_dim_caps=1024,
                                          out_num_caps=num_classes, out_dim_caps=16,
                                          routings=3)
        elif (self.configuration.model_name == "xlnet-base-cased"):
            self.digitcaps = DenseCapsule(in_num_caps=in_num_caps,
                                          in_dim_caps=768,
                                          out_num_caps=num_classes, out_dim_caps=16,
                                          routings=3)

        self.dense = torch.nn.Linear(num_classes * 16, num_classes * 16)
        self.softmax = torch.nn.Softmax()
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(num_classes * 16, num_classes)

    def forward(
            self,
            input_ids=None,
            attention_mask=None,
            token_type_ids=None,
            position_ids=None,
            head_mask=None,
            inputs_embeds=None,
            class_label=None,
            exclusion_mask= None
    ):

        content_output = self.bert_model_content(input_ids, attention_mask=attention_mask)

        if exclusion_mask is not None and self.configuration.CONDITIONAL_TRAINING:
            content_output = content_output[0] * exclusion_mask[0].unsqueeze(dim=-1)
        else:
            content_output = content_output[0]

        if self.configuration.USE_SEQUENCE_SUMMARY:
            output_summary = self.sequence_summary(content_output)
            content_output = torch.cat([content_output, output_summary.unsqueeze(dim=1)], dim=1)


        if self.configuration.SQUASH_AFTER_TRANSFORMER:
            op_bert_ensemble = squash(content_output)
        else:
            op_bert_ensemble = content_output
        
        
        classvecs = self.digitcaps(op_bert_ensemble)

        if self.configuration.CAPS_PURE:
            outputs = classvecs.norm(dim=-1)
        else:
            classvecs_fl = classvecs.view(input_ids.size(0), -1)
            if self.configuration.SOFTMAX_DENSE_LAYER:
                x = self.softmax(self.dense(classvecs_fl))
            else:
                x = self.dense(classvecs_fl)
            x = self.dropout(x)
            outputs = self.classifier(x)

        return outputs

In [11]:
class BertEnsembleClassifier(object):
    '''
    classdocs
    '''

    def __init__(self, base_dir, modelOverRideForEval = None,  configuration=None, mode="eval", cross=0):
        '''
        Constructor
        '''
        if configuration:
            self.configuration = configuration
        else:
            self.configuration = BertEnsembleModelConfig()
        self.BASE_DIR = base_dir
        self.labelEncoderPath = os.path.join(self.BASE_DIR, self.configuration.labelEncoderFileName)
        self.modelPath = os.path.join(self.BASE_DIR, self.configuration.savedModelFileName)
        self.device = 'cuda' if cuda.is_available() else 'cpu'
        self.cross = cross

        if self.configuration.DETERMINISTIC:
            self.seed_everything()

        self.tokenizer = self.configuration.tokenizer
        if mode == "eval":
            self.lb = joblib.load(self.labelEncoderPath)
            self.model = BertEnsembletModel(len(self.lb.classes_), self.configuration)
            if modelOverRideForEval:
                self.model.load_state_dict(torch.load(modelOverRideForEval, map_location=self.device))
                self.model.eval()
            else:
                modelCheckpointer = ModelCheckPointer()
                modelCheckpointer.loadBestModel(self.BASE_DIR, self.model, self.device)
                self.model.eval()
        elif mode == "train":
            self.avg_train_losses = []
            self.avg_valid_losses = []
            self.train_accuracy_list = []
            self.valid_accuracy_list = []
            self.LR = []
            self.DEBUG_INCORRECT = {}


        else:
            raise Exception("illegal mode")

    def predict(self, content, processed=False):

        self.model.eval()
        if not processed:
            content = cleanSingleSentenceWithoutRemovingMeaning(content)

        content = " ".join(content.split(" ")[0:self.configuration.MAX_LEN])

        with torch.no_grad():
            inputs = self.tokenizer.encode_plus(
                content,
                None,
                add_special_tokens=True,
                max_length=self.configuration.MAX_LEN,
                pad_to_max_length=True,
                return_attention_mask=True,
                # return_tensors='pt',
                truncation=True
            )
            ids = inputs['input_ids']
            mask = inputs['attention_mask']

            ids = [ids]
            mask = [mask]

            contentIds = torch.tensor(ids, dtype=torch.long)
            contentMask = torch.tensor(mask, dtype=torch.long)
            contenteExclusionMask = torch.ones((1))

            inputs = {
                "input_ids": contentIds,
                "attention_mask": contentMask,
                "exclusion_mask": contenteExclusionMask
            }

            outputs = self.model(**inputs)
            prediction = torch.sigmoid(outputs)

        return self.lb.inverse_transform(prediction.detach().numpy())

    def seed_everything(self, seed=1234):
        random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        np.random.seed(seed)
        os.environ['PYTHONHASHSEED'] = str(seed)
        torch.backends.cudnn.deterministic = True

    def loss_fn(self, outputs, targets):
        if self.configuration.LOSS_FN == "CrossEntropyLoss":
            torch.nn.CrossEntropyLoss()(outputs, targets)
        elif self.configuration.LOSS_FN == "BCEWithLogitsLoss":
            return torch.nn.BCEWithLogitsLoss()(outputs, targets)
        elif self.configuration.LOSS_FN == "MarginLoss":
            return self.caps_loss(targets, outputs)
        elif self.configuration.LOSS_FN == "FocalLoss":
            return FocalLoss()(outputs, targets)

    def caps_loss(self, y_true, y_pred):
        """
        Capsule loss = Margin loss
        :param y_true: true labels, one-hot coding, size=[batch, classes]
        :param y_pred: predicted labels by CapsNet, size=[batch, classes]
        :return: Variable contains a scalar loss value.
        """
        L = y_true * torch.clamp(0.9 - y_pred, min=0.) ** 2 + \
            0.5 * (1 - y_true) * torch.clamp(y_pred - 0.1, min=0.) ** 2
        L_margin = L.sum(dim=1).mean()

        return L_margin

    def run_evaluation(self, model, validation_data_loader_content):
        model.eval()
        valid_losses = []
        accuracyBoolList = []
        confusionMatrix = torch.zeros(len(self.lb.classes_), len(self.lb.classes_))
        inncorrectPreds = []

        with torch.no_grad():
          for step, batch in tqdm(enumerate(validation_data_loader_content), desc="running evaluation"):
              contentBatch = batch
              batch_1 = tuple(t.to(self.device) for t in contentBatch)

              targets = batch_1[2]
              inputs = {
                  "input_ids": batch_1[0],
                  "attention_mask": batch_1[1],
                  "exclusion_mask": batch_1[3]
              }
              outputs = model(**inputs)
              loss = self.loss_fn(outputs, targets)
              valid_losses.append(loss.item())
              _, predicted = torch.max(outputs, 1)
              _, trueClass = torch.max(targets, 1)
              for trueClassLabel, predictedClassLabel in zip(trueClass, predicted):
                  confusionMatrix[trueClassLabel, predictedClassLabel] = confusionMatrix[trueClassLabel, predictedClassLabel] +1
              booleans = (predicted == trueClass)
              accuracyBoolList.extend([boolean.item() for boolean in booleans])
              
              # print("predicted")
              # print(predicted)
              # print(trueClass)

              
              incorrectPredictionIndices = [i for i, x in enumerate(booleans) if not x]
              for index in incorrectPredictionIndices:
                inncorrectPreds.append(batch_1[0][index].detach().cpu().numpy())
                  
                  

              del outputs, inputs, batch_1
              gc.collect()
        return valid_losses, accuracyBoolList, confusionMatrix, inncorrectPreds

    def run_training(self, epoch, model, training_data_loader_content, optimizer, scheduler):

        model.train()
        model.zero_grad()
        train_losses = []
        accuracyBoolList = []
        confusionMatrix = torch.zeros(len(self.lb.classes_), len(self.lb.classes_))

        for step, batch in tqdm(enumerate(training_data_loader_content), desc="running training for epoch {}".format(epoch)):
            contentBatch = batch

            batch_1 = tuple(t.to(self.device) for t in contentBatch)

            targets = batch_1[2]
            inputs = {
                "input_ids": batch_1[0],
                "attention_mask": batch_1[1],
                "exclusion_mask": batch_1[3]
            }
            outputs = model(**inputs)
            loss = self.loss_fn(outputs, targets)
            train_losses.append(loss.item())
            if self.configuration.ACCUMULATION_STEPS != 1:
                loss = loss / self.configuration.ACCUMULATION_STEPS
            loss.backward()

            _, predicted = torch.max(outputs, 1)
            _, trueClass = torch.max(targets, 1)
            for trueClassLabel, predictedClassLabel in zip(trueClass, predicted):
                confusionMatrix[trueClassLabel, predictedClassLabel] = confusionMatrix[
                                                                           trueClassLabel, predictedClassLabel] + 1
            booleans = (predicted == trueClass)
            accuracyBoolList.extend([boolean.item() for boolean in booleans])

            if (step + 1) % self.configuration.ACCUMULATION_STEPS == 0:
                if self.configuration.max_grad_norm is not None:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), self.configuration.max_grad_norm)
                optimizer.step()  # Now we can do an optimizer step
                if self.configuration.LR_DECAY_MODE == "BATCH" and self.configuration.LEARNING_RATE_AUTO_DECAY_FLAG:
                    print("Learning rate decay at batch level, reducing LR")
                    scheduler.step()
                    self.LR.append(scheduler.get_lr())
                #optimizer.zero_grad()
                model.zero_grad()
                print('Epoch: {}, step: {},  Loss:  {}'.format(epoch, step, loss.item()))


            del outputs, inputs, batch_1
            gc.collect()
        return train_losses, accuracyBoolList, confusionMatrix

    def train(self, training_data, testing_data, trainDataSize, trainFromScratch=True):

        self.lb = LabelBinarizer()
        training_data['labelvec'] = self.lb.fit_transform(training_data['Mood']).tolist()

        training_data_content = training_data[['lyrics', 'labelvec']]
        training_data_content = training_data_content.rename(columns={"lyrics": "text"})
        joblib.dump(self.lb, self.labelEncoderPath)

        testing_data['labelvec'] = self.lb.transform(testing_data['Mood']).tolist()

        testingDataContent = testing_data[['lyrics', 'labelvec']]
        testingDataContent = testingDataContent.rename(columns={"lyrics": "text"})

        print("creatinng dataset. The tokenizer is ", self.tokenizer)
        content_training_set = CustomDataset(training_data_content, self.tokenizer, self.configuration.MAX_LEN)
        content_testing_set = CustomDataset(testingDataContent, self.tokenizer, self.configuration.MAX_LEN)


        content_training_loader = DataLoader(content_training_set,
                                             batch_size=self.configuration.TRAIN_BATCH_SIZE[0],
                                             sampler=SequentialSampler(content_training_set), drop_last=False)


        content_testing_loader = DataLoader(content_testing_set,
                                               batch_size=self.configuration.VALID_BATCH_SIZE,
                                               sampler=SequentialSampler(content_testing_set), drop_last=False)


        model = BertEnsembletModel(len(self.lb.classes_))

        no_decay = ["bias", "LayerNorm.weight"]
        if self.configuration.WEIGHT_DECAY == 0.0:
            print("weight decay parameter is 0 so, using no weight decay anywhere")
            optimizer_grouped_parameters = [{
                "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            }]
        else:
            print("weight decay parameter is {} so, using this weight decay anywhere".format(self.configuration.WEIGHT_DECAY))
            optimizer_grouped_parameters = [
                {
                    "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                    "weight_decay": self.configuration.WEIGHT_DECAY,
                },
                {
                    "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                    "weight_decay": 0.0,
                },
            ]

        if self.configuration.LR_DECAY_MODE == "BATCH":
            t_total = len(content_training_loader) // self.configuration.ACCUMULATION_STEPS * self.configuration.EPOCHS
            print("Learning rate decay at batch level, t_total is {}".format(t_total))
        elif self.configuration.LR_DECAY_MODE == "EPOCH":
            t_total = self.configuration.EPOCHS
            print("Learning rate decay at epoch level, t_total is {}".format(t_total))

        if self.configuration.WARM_UP_STEPS == None:
            warmup_steps = math.ceil(t_total * self.configuration.WARM_UP_RATIO)
        else:
            warmup_steps = self.configuration.WARM_UP_STEPS
        optimizer = torch.optim.AdamW(params=optimizer_grouped_parameters, lr=self.configuration.LEARNING_RATE)
        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total
        )

        savedEpoch = 0
        if not trainFromScratch:
            # PATH = previousSavedModel
            modelCheckpointer = ModelCheckPointer()
            savedEpoch = modelCheckpointer.load_checkpoint(self.BASE_DIR, model, self.device, optimizer, scheduler)
            # model.load_state_dict(torch.load(PATH, map_location=self.device))
        model.to(self.device)

        early_stopping = EarlyStoppingAndCheckPointer(patience=self.configuration.PATIENCE, verbose=True, basedir=self.BASE_DIR, epoch_level_save=False)

        #self.initialLog(model,content_training_loader, content_testing_loader)
        for epoch in range(savedEpoch, self.configuration.EPOCHS):
            print("starting training. The LR is {}".format(scheduler.get_lr()))

            trainBatchSizeForEpoch = self.configuration.TRAIN_BATCH_SIZE[epoch]
            if trainBatchSizeForEpoch < 1:
                trainBatchSizeForEpoch = 1
            content_training_loader = DataLoader(content_training_set,
                                                 batch_size=trainBatchSizeForEpoch,
                                                 sampler=SequentialSampler(content_training_set), drop_last=False)


            if not self.configuration.LEARNING_RATE_AUTO_DECAY_FLAG:
                lrForEpoch = self.configuration.LEARNING_RATE * self.configuration.LERANING_RATE_DECAY_MANUAL[epoch]
                for param_group in optimizer.param_groups:
                    param_group['lr'] = lrForEpoch
                    print("setting LR manually to {}".format(lrForEpoch))

            train_losses, accuracyBoolListTrain, confusionMatrixTrain = self.run_training(epoch, model, content_training_loader, optimizer, scheduler)
            if self.configuration.LR_DECAY_MODE == "EPOCH" and self.configuration.LEARNING_RATE_AUTO_DECAY_FLAG:
                scheduler.step()
                print("Learning rate decay at epoch level, reducing LR to {}".format(scheduler.get_lr()))
                self.LR.append(scheduler.get_lr())

            valid_losses, accuracyBoolListValid, confusionMatrixValid, inncorrectPreds = self.run_evaluation(model, content_testing_loader)

            train_loss = np.average(train_losses)
            valid_loss = np.average(valid_losses)
            accuracy_train = sum(accuracyBoolListTrain)/len(accuracyBoolListTrain) * 100
            accuracy_valid = sum(accuracyBoolListValid) / len(accuracyBoolListValid) * 100

            self.avg_train_losses.append(train_loss)
            self.avg_valid_losses.append(valid_loss)
            self.train_accuracy_list.append(accuracy_train)
            self.valid_accuracy_list.append(accuracy_valid)

            print('Epoch: {},  Total Train Loss:  {}'.format(epoch+1, train_loss))
            print('Epoch: {},  Total Validation Loss:  {}'.format(epoch+1, valid_loss))
            print('Epoch: {},  Total training accuracy:  {}'.format(epoch+1, accuracy_train))
            print('Epoch: {},  Total Validation accuracy:  {}'.format(epoch+1, accuracy_valid))

            early_stopping(valid_loss, model, optimizer, epoch, scheduler)


            self.visualizeTraining(epoch+1, confusionMatrixTrain, cross = self.cross)
            self.visualizeValAccuracy(epoch+1, confusionMatrixValid, cross = self.cross)
            self.visualizeLR(epoch + 1,  cross = self.cross)
            if early_stopping.early_stop:
                print("Early stopping")
                self.model = model
                break

    def initialLog(self, model,
                   content_training_loader,
                   content_testing_loader):
        train_losses, accuracyBoolListTrain, confusionMatrixTrain = self.run_evaluation(model,content_training_loader)
        valid_losses, accuracyBoolListValid, confusionMatrixValid = self.run_evaluation(model,content_testing_loader)

        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        accuracy_train = sum(accuracyBoolListTrain) / len(accuracyBoolListTrain) * 100
        accuracy_valid = sum(accuracyBoolListValid) / len(accuracyBoolListValid) * 100

        self.avg_train_losses.append(train_loss)
        self.avg_valid_losses.append(valid_loss)
        self.train_accuracy_list.append(accuracy_train)
        self.valid_accuracy_list.append(accuracy_valid)
        self.visualizeTraining(0, confusionMatrixTrain,  cross = self.cross)
        self.visualizeValAccuracy(0, confusionMatrixValid,  cross = self.cross)

    def visualizeTraining(self, epoch, confusionMatrixTrain,  cross = 0):
        # visualize the loss as the network trained
        fig = plt.figure(figsize=(10, 8))
        plt.plot(range(0, len(self.avg_train_losses)), self.avg_train_losses, label='Training Loss')
        plt.plot(range(0, len(self.avg_valid_losses)), self.avg_valid_losses, label='Validation Loss')
        minposs = self.avg_valid_losses.index(min(self.avg_valid_losses))
        plt.axvline(minposs, linestyle='--', color='r', label='Early Stopping Checkpoint')


        plt.xlabel('epochs')
        plt.ylabel('loss')
        plt.ylim(0, 0.5)  # consistent scale
        plt.xlim(0, len(self.avg_train_losses))  # consistent scale
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()
        fig.savefig(os.path.join(self.BASE_DIR , 'loss_plot_cross-{}_epoch-{}.png'.format(cross, epoch)), bbox_inches='tight')

        fig = plt.figure(figsize=(10, 8))
        confusionMatrixTrain = confusionMatrixTrain.numpy()
        confusionMatrixTrain = confusionMatrixTrain / confusionMatrixTrain.astype(np.float).sum(axis=1, keepdims=True)
        hmap = sns.heatmap(confusionMatrixTrain, annot=True,
                           fmt='.2', cmap='Blues', annot_kws={"size": 6},xticklabels=self.lb.classes_, yticklabels=self.lb.classes_)
        hmap.set_xticklabels(hmap.get_xmajorticklabels(), fontsize=6)
        hmap.set_yticklabels(hmap.get_ymajorticklabels(), fontsize=6)
        figure = hmap.get_figure()
        figure.savefig(os.path.join(self.BASE_DIR ,'training_confusion_matrix_cross-{}_epoch-{}.png'.format(cross, epoch)), dpi=400)

    def visualizeValAccuracy(self, epoch, confusionMatrix,  cross = 0):
        # visualize the loss as the network trained
        fig = plt.figure(figsize=(10, 8))
        plt.plot(range(0, len(self.valid_accuracy_list)), self.valid_accuracy_list, label='Validation Accuracy')
        plt.plot(range(0, len(self.train_accuracy_list)), self.train_accuracy_list, label='Training Accuracy')

        # find position of lowest validation loss

        plt.xlabel('epochs')
        plt.ylabel('accuracy')
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()
        fig.savefig(os.path.join(self.BASE_DIR , 'accuracy_plot_cross-{}_epoch-{}.png'.format(cross, epoch)), bbox_inches='tight')

        fig = plt.figure(figsize=(10, 8))
        confusionMatrix = confusionMatrix.numpy()
        confusionMatrix = confusionMatrix / confusionMatrix.astype(np.float).sum(axis=1, keepdims=True)
        hmap = sns.heatmap(confusionMatrix , annot=True,
                    fmt='.2', cmap='Blues', annot_kws={"size": 6},xticklabels=self.lb.classes_, yticklabels=self.lb.classes_)
        hmap.set_xticklabels(hmap.get_xmajorticklabels(), fontsize=6)
        hmap.set_yticklabels(hmap.get_ymajorticklabels(), fontsize=6)
        figure = hmap.get_figure()
        figure.savefig(os.path.join(self.BASE_DIR ,'validation_confusion_matrix_cross-{}_epoch-{}.png'.format(cross, epoch)), dpi=400)


    def visualizeLR(self, epoch, cross=0):
        # visualize the loss as the network trained
        fig = plt.figure(figsize=(10, 8))
        plt.plot(range(0, len(self.LR)), self.LR, label='Learning rate')

        # find position of lowest validation loss

        if self.configuration.LR_DECAY_MODE == "EPOCH":
            plt.xlabel('epochs')
        else:
            plt.xlabel('step')
        plt.ylabel('learning rate')
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()
        fig.savefig(os.path.join(self.BASE_DIR , 'lr_plot_cross-{}_epoch-{}.png'.format(cross, epoch)), bbox_inches='tight')

In [12]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        print("inn dataset init", tokenizer)
        self.tokenizer = tokenizer
        print("inn dataset init", self.tokenizer)
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.labelvec
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split()[0:self.max_len])

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_attention_mask=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        exclusionMask = torch.ones((1))

        return torch.tensor(ids, dtype=torch.long), torch.tensor(mask, dtype=torch.long), torch.tensor(
            self.targets[index], dtype=torch.float), exclusionMask

In [ ]:
if __name__ == '__main__':
    runMode = "train"
    cross_Fold = False
    BASE_DIR = "/content/drive/My Drive/ICMPS/bert_mood_capsule_content"
    if(runMode=="train"):
        from google.colab import drive
        drive.mount('/content/drive', force_remount=True)

              
        if not os.path.exists(BASE_DIR):
            os.mkdir(BASE_DIR)

        originalData = pd.read_pickle(os.path.join(BASE_DIR, "ml_balanced_data_augmented.pkl"))

        originalData = originalData.replace(np.nan, '', regex=True)

        #originalData['lyrics'] = originalData['lyrics'].apply(lambda x: cleanSingleSentenceWithoutRemovingMeaning(x))

        originalData = originalData[originalData['lyrics'] != ""]

        originalData = originalData.reset_index(drop=True)

        skfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=3)
        val_acc_list = []
        t = originalData.Mood
        cross = 0
        for train_index, test_index in skfold.split(np.zeros(len(t)), t):
            train = originalData.iloc[train_index]
            test = originalData.iloc[test_index]

            train = train.reset_index(drop=True)
            test = test.reset_index(drop=True)

            bert_ensemble_model = BertEnsembleClassifier(BASE_DIR, mode="train", cross=cross)
            bert_ensemble_model.train(train, test, len(train.index), trainFromScratch=True)
            print("After Training of the current fold - validation accuracy {}".format(bert_ensemble_model.valid_accuracy_list))
            val_acc_list.append(bert_ensemble_model.valid_accuracy_list)

            if not cross_Fold:
              break


        print("final list {}".format(val_acc_list))
        accSum = []
        for foldAcc in val_acc_list:
            accSum.append(max(foldAcc))

        print("max list {}".format(accSum))
        print("average accuracy {}".format(np.average(accSum)))




    else:
        pass